# Intro to the Allen Brain Observatory Visual Coding Ophys Dataset.
This notebook demonstrates how to access and visualize data.

In [ ]:
!apt install s3fs
!pip install allensdk
!mkdir -p /data/allen-brain-observatory/
!s3fs allen-brain-observatory /data/allen-brain-observatory/ -o public_bucket=1



Standard Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Allen Brain Observatory set up
This instantiates the tools in the Allen SDK that allow us to access the Brain Observatory data.

The main entry point is the `BrainObservatoryCache` class. This class is responsible for accessing any data or metadata.

We begin by importing the `BrainObservatoryCache` class and instantiating it, pointing it to our manifest file.

`manifest_file` is a path to where the manifest file is located. This needs to reflect where you are storing and accessing the data. Here, we are pointing it to the data on the S3 bucket.

In [ ]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
manifest_file = '/data/allen-brain-observatory/visual-coding-2p/manifest.json'
boc = BrainObservatoryCache(manifest_file=manifest_file)

### Querying from the Brain Observatory Cache

The Brain Observatory Cache enables us to see the dimensions of the dataset.

Let's take a look at the available depths, Cre lines, areas, and stimuli available in the Brain Observatory datsset.



In [ ]:
###########
# list of all targeted areas


In [ ]:
###########
# list of all Cre driver lines


In [ ]:
###########
# list of all imaging depths


In [ ]:
###########
# list of all stimuli


### Finding Experiment containers in the Brain Observatory Cache
The `experiment container` describes a set of 3 `sessions` performed for the same field of view (ie. same targeted area and imaging depth in the same mouse that targets the same set of neurons). Each experiment container has a unique ID number.

!['Diagram of containers'](http://alleninstitute.github.io/AllenSDK/_static/container_session_layout.png)

First let's try to find all the experiment containers:

In [ ]:
########


And now let's find all the "sessions" (These are Ophys Experiment Sessions):

In [ ]:
#########


#### Finding Experiment Containers of Interest

We may be interested in a specific visual area and Cre line. Let's choose a visual area and Cre line from the lists above.

In [ ]:
visual_area = 'VISal'
cre_line ='Cux2-CreERT2'

Get the list of all the experiment containers for that area and Cre line combination.

In [ ]:
? boc.get_experiment_containers

In [ ]:
###############


A nice way to look at it is by using a Pandas Dataframe:

In [ ]:
pd.DataFrame(exps)

How many experiment containers did you find?

In [ ]:
len(exps)

Try this: Let's look at one experiment container, imaged from Cux2-CreERT2, in VISp, from imaging depth 175 um. Compute the DataFrame that will show you all the experiment containers for this combination of parameters.

In [ ]:
##################



Now let's look at one of these experiment containers:

In [ ]:
experiment_container_id = 511510736

Let's get all of the sessions for this container.

In [ ]:
?boc.get_ophys_experiments

In [ ]:
#############



Let's find the session from this container that used the `natural_scenes` stimulus.

In [ ]:
#############




Each session has a unique ID, and we can use that ID to access the data for this session.

In [ ]:
session_id = boc.get_ophys_experiments(experiment_container_ids=[experiment_container_id],
                                       stimuli=['natural_scenes'])[0]['id']
print(session_id)

This concludes the section on querying the Brain Observatory Cache for searching for experiments. We will now look into how we can find information about specific Sessions (Ophys Experiments)

# Ophys Experiment data
A single imaging session is stored in an NWB File. This section shows us how to access everything in the NWB file for a single imaging session. An ophys session contains the following:


1.   Maximum Projection of the Ophys Session
2.   ROI masks for cells
3.   DF/F Traces
4.   Stimulus Epochs
5.   Running Speed
6.   Stimulus Table
7.   Stimulus Template



In [ ]:
?boc.get_ophys_experiment_data

In [ ]:
#######


We can use this `data_set` object to access all the pieces of data for the session. Let's take a look.

# Maximum projection
This is the projection of the full motion corrected movie. It shows all of the neurons imaged during the session.

In [ ]:
###########


In [ ]:


fig = plt.figure(figsize=(6,6))
plt.imshow(max_projection, cmap='gray')

# ROI Masks
These are all of the segmented masks for cell bodies in this experiment.

In [ ]:
#########


Look at the dimensions of the mask. There is one image per cell.

In [ ]:
print(rois.shape)
print("Number of cells:", rois.shape[0])

What are the values in the mask?

In [ ]:
################


Let's look at a mask for one cell:

In [ ]:
################



To look at all cells, instead we can sum them up:

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(rois.sum(axis=0))

In [ ]:
np.unique(rois.sum(axis=0))

# DF/F Traces
There are a number of accessible traces for all these 174 cells in the NWB file, including raw fluorescence, neuropil corrected traces, demixed traces, and DF/F traces. There are also extracted events available.
In this tutorial we will us DF/F to examine neural activity.

In [ ]:
##############


Let's look at dff and ts. (dff is the stored variable name for the DF/F traces and ts are the timestamps.)

In [ ]:
##############


In [ ]:
#############


Let's plot the activity of one neuron:

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(dff[10,:], color='gray')

Let's plot the activity of the first 50 neurons from this session.

In [ ]:
fig = plt.figure(figsize=(10,8))
for i in range(50):
    plt.plot(dff[i,:])
    #plt.plot(dff[i,:]+(i*2), color='gray')

# Stimulus epochs
Several stimuli are shown during each imaging session, interleaved with each other. The stimulus epoch table provides information on these interleaved stimulus epochs

In [ ]:
############


Let's add the stimulus epoch information to the plot of neural activity.

In [ ]:
#plot activity of first 50 neurons
fig = plt.figure(figsize=(10,8))
for i in range(50):
    plt.plot(dff[i,:]+(i*2), color='gray')

#for each stimulus, shade the plot when the stimulus is presented
colors = ['blue','orange','green','red']
for c,stim_name in enumerate(stim_epoch.stimulus.unique()):
    stim = stim_epoch[stim_epoch.stimulus==stim_name]
    for j in range(len(stim)):
        plt.axvspan(xmin=stim.start.iloc[j], xmax=stim.end.iloc[j], color=colors[c], alpha=0.1)


# Running speed
The running speed of the animal on the rotating disk during the entire session.

In [ ]:
dxcm, tsd = data_set.get_running_speed()

fig = plt.figure(figsize=(10,3))
plt.plot(dxcm)
plt.ylabel("Running speed (cm/s)")

Let's add the running speed to our plot of neural activity and stimulus epochs.

In [ ]:
#plot activity of first 50 neurons
fig = plt.figure(figsize=(10,10))
for i in range(50):
    plt.plot(dff[i,:]+(i*2), color='gray')

#plot the running speed (scaled and offset to fit)
plt.plot((0.2*dxcm)-20)

#for each stimulus, shade the plot when the stimulus is presented
colors = ['blue','orange','green','red']
for c,stim_name in enumerate(stim_epoch.stimulus.unique()):
    stim = stim_epoch[stim_epoch.stimulus==stim_name]
    for j in range(len(stim)):
        plt.axvspan(xmin=stim.start.iloc[j], xmax=stim.end.iloc[j], color=colors[c], alpha=0.1)


Let's look at a few individual neurons.

In [ ]:
fig = plt.figure(figsize=(14,4))

plt.plot(dff[49,:], color='gray')
plt.plot((0.1*dxcm)-10)

#for each stimulus, shade the plot when the stimulus is presented
colors = ['blue','orange','green','red']
for c,stim_name in enumerate(stim_epoch.stimulus.unique()):
    stim = stim_epoch[stim_epoch.stimulus==stim_name]
    for j in range(len(stim)):
        plt.axvspan(xmin=stim.start.iloc[j], xmax=stim.end.iloc[j], color=colors[c], alpha=0.1)


In [ ]:
fig = plt.figure(figsize=(14,4))

plt.plot(dff[4,:], color='gray')
plt.plot((0.1*dxcm)-10)

#for each stimulus, shade the plot when the stimulus is presented
colors = ['blue','orange','green','red']
for c,stim_name in enumerate(stim_epoch.stimulus.unique()):
    stim = stim_epoch[stim_epoch.stimulus==stim_name]
    for j in range(len(stim)):
        plt.axvspan(xmin=stim.start.iloc[j], xmax=stim.end.iloc[j], color=colors[c], alpha=0.1)

In [ ]:
fig = plt.figure(figsize=(14,4))

plt.plot(dff[35,:], color='gray')
plt.plot((0.1*dxcm)-10)

#for each stimulus, shade the plot when the stimulus is presented
colors = ['blue','orange','green','red']
for c,stim_name in enumerate(stim_epoch.stimulus.unique()):
    stim = stim_epoch[stim_epoch.stimulus==stim_name]
    for j in range(len(stim)):
        plt.axvspan(xmin=stim.start.iloc[j], xmax=stim.end.iloc[j], color=colors[c], alpha=0.1)

# Stimulus Table
For each stimulus there is a stimulus table with information about the condition and timing of each trial.

In [ ]:
natural_scene_table = data_set.get_stimulus_table('natural_scenes')
natural_scene_table.head(n=10)


# Stimulus Template
The images and movies presented during the session area also included in the NWB file as the stimulus template. Stimuli that are generated programmatically (eg. drifting and static gratings) do not have a stimulus template. There are tools in the SDK to recreate these stimuli.

In [ ]:
natural_scene_template = data_set.get_stimulus_template('natural_scenes')

In [ ]:
natural_scene_template.shape

Let's look at the scene presented in the first trial.

In [ ]:
scene_number = natural_scene_table.frame.loc[0]
plt.imshow(natural_scene_template[scene_number,:,:], cmap='gray')

We can add the trials of this image to the plot of neural activity too:

In [ ]:
#plot activity of first 50 neurons
fig = plt.figure(figsize=(10,10))
for i in range(50):
    plt.plot(dff[i,:]+(i*2), color='gray')

#plot the running speed (scaled and offset to fit)
plt.plot((0.2*dxcm)-20)

#for each stimulus, shade the plot when the stimulus is presented
colors = ['blue','orange','green','red']
for c,stim_name in enumerate(stim_epoch.stimulus.unique()):
    stim = stim_epoch[stim_epoch.stimulus==stim_name]
    for j in range(len(stim)):
        plt.axvspan(xmin=stim.start.iloc[j], xmax=stim.end.iloc[j], color=colors[c], alpha=0.1)

#shade traces with the time of each presentation of the above scene
stim_subset = natural_scene_table[natural_scene_table.frame==scene_number]
for j in range(len(stim_subset)):
    plt.axvspan(xmin=stim_subset.start.iloc[j], xmax=stim_subset.end.iloc[j], color='red', alpha=0.4)

We can zoom in on these trials. We'll look at one neuron's responses to all the trials of one image.

In [ ]:
cell_index=19
scene_number=22

stim_subset = natural_scene_table[natural_scene_table.frame==scene_number]

for i in range(len(stim_subset)):
    plt.plot(dff[cell_index,stim_subset.start.iloc[i]-10:stim_subset.end.iloc[i]+10], color='gray')
plt.axvspan(10,18, color='red',alpha=0.2)
